# *Starting to Crawl*

Web crawler are called such because they crawl across the Web. They must retrieve, in a recursive manner, page contents for a URL, examine that page for another URL, and retrieve that page, ad infinitum.

## 1. Traversing a Single Domain

Write a Python script that retrieves an arbitrary Wikipedia page and produces a list of links on that page:

In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

html = urlopen("http://en.wikipedia.org/wiki/Kevin_Bacon")
bsObj = BeautifulSoup(html)
for link in bsObj.findAll("a"):
    if 'href' in link.attrs:
        print(link.attrs['href'])

/wiki/Wikipedia:Protection_policy#semi
#mw-head
#searchInput
/wiki/Kevin_Bacon_(disambiguation)
/wiki/File:Kevin_Bacon_SDCC_2014.jpg
/wiki/Philadelphia,_Pennsylvania
/wiki/Kyra_Sedgwick
/wiki/Sosie_Bacon
#cite_note-1
/wiki/Edmund_Bacon_(architect)
/wiki/Michael_Bacon_(musician)
/wiki/Holly_Near
/wiki/Wikipedia:Citation_needed
http://baconbros.com/
#cite_note-2
#cite_note-actor-3
/wiki/Footloose_(1984_film)
/wiki/JFK_(film)
/wiki/A_Few_Good_Men
/wiki/Apollo_13_(film)
/wiki/Mystic_River_(film)
/wiki/Balto_(film)
/wiki/Sleepers
/wiki/The_Woodsman_(2004_film)
/wiki/Animal_House
/wiki/Diner_(1982_film)
/wiki/Tremors_(1990_film)
/wiki/Crazy,_Stupid,_Love
/wiki/Friday_the_13th_(1980_film)
/wiki/Flatliners
/wiki/The_River_Wild
/wiki/Wild_Things_(film)
/wiki/Stir_of_Echoes
/wiki/Hollow_Man
/wiki/Frost/Nixon_(film)
/wiki/X-Men:_First_Class
/wiki/Black_Mass_(film)
/wiki/Patriots_Day_(film)
/wiki/Fox_Broadcasting_Company
/wiki/The_Following
/wiki/HBO
/wiki/Taking_Chance
/wiki/Golden_Globe_Award
/w

But.. we are serching links that point to article pages:

In [5]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

html = urlopen("http://en.wikipedia.org/wiki/Kevin_Bacon")
bsObj = BeautifulSoup(html)
for link in bsObj.find("div",{"id":"bodyContent"}).findAll("a",href=re.compile("^(/wiki/)((?!:).)*$")):
    if 'href' in link.attrs:
        print(link.attrs['href'])

/wiki/Kevin_Bacon_(disambiguation)
/wiki/Philadelphia,_Pennsylvania
/wiki/Kyra_Sedgwick
/wiki/Sosie_Bacon
/wiki/Edmund_Bacon_(architect)
/wiki/Michael_Bacon_(musician)
/wiki/Holly_Near
/wiki/Footloose_(1984_film)
/wiki/JFK_(film)
/wiki/A_Few_Good_Men
/wiki/Apollo_13_(film)
/wiki/Mystic_River_(film)
/wiki/Balto_(film)
/wiki/Sleepers
/wiki/The_Woodsman_(2004_film)
/wiki/Animal_House
/wiki/Diner_(1982_film)
/wiki/Tremors_(1990_film)
/wiki/Crazy,_Stupid,_Love
/wiki/Friday_the_13th_(1980_film)
/wiki/Flatliners
/wiki/The_River_Wild
/wiki/Wild_Things_(film)
/wiki/Stir_of_Echoes
/wiki/Hollow_Man
/wiki/Frost/Nixon_(film)
/wiki/Black_Mass_(film)
/wiki/Patriots_Day_(film)
/wiki/Fox_Broadcasting_Company
/wiki/The_Following
/wiki/HBO
/wiki/Taking_Chance
/wiki/Golden_Globe_Award
/wiki/Screen_Actors_Guild_Award
/wiki/Primetime_Emmy_Award
/wiki/Streaming_television
/wiki/I_Love_Dick_(TV_series)
/wiki/Golden_Globe_Award_for_Best_Actor_%E2%80%93_Television_Series_Musical_or_Comedy
/wiki/The_Guardian
/wi

Having a script that finds all article links in one is fairly useless in practice. We need a code able to:<br>
    - Get a list a off all linked articles <br>
    - Starting from a random article and jump among related links calling at each step the *getLinks* function.

In [10]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import datetime
import random
import re

random.seed(datetime.datetime.now())

def getLinks(articleUrl):
    #Extract all internal links in a page related to a specific articleUrl
    html = urlopen("http://en.wikipedia.org"+articleUrl)
    bsObj = BeautifulSoup(html)
    return bsObj.find("div",{"id":"bodyContent"}).findAll("a",href=re.compile("^(/wiki/)((?!:).)*$"))

links = getLinks("/wiki/Kevin_Bacon")
while len(links) > 0 :
    newArticle = links[random.randint(0, len(links)-1)].attrs["href"]
    print(newArticle)
    links = getLinks(newArticle)

/wiki/Taking_Chance
/wiki/Producers_Guild_of_America_Award_for_Best_Long-Form_Television
/wiki/The_Abominable_Bride
/wiki/Behind_the_Candelabra
/wiki/Solitary_Man_(film)
/wiki/Roger_Ebert
/wiki/Carotid_artery
/wiki/Internal_carotid_artery
/wiki/Orbitofrontal_artery
/wiki/Greater_palatine_artery
/wiki/Frontal_branch_of_superficial_temporal_artery
/wiki/Accessory_meningeal_artery
/wiki/Descending_palatine_artery
/wiki/Glandular_branches_of_the_superior_thyroid_artery
/wiki/Meningeal_branch_of_occipital_artery
/wiki/Dorsal_nasal_artery
/wiki/Caroticotympanic_arteries
/wiki/Inferior_hypophysial_artery
/wiki/Anterior_ethmoidal_artery
/wiki/Acromial_branch_of_suprascapular_artery
/wiki/Anterior_auricular_branches_of_superficial_temporal_artery
/wiki/ISSN_(identifier)
/wiki/Photographic_Activity_Test
/wiki/IATF_16949
/wiki/Open_Data_Protocol
/wiki/IEC_63382
/wiki/OpenDocument
/wiki/X.500
/wiki/ANSI_C
/wiki/C_string_handling
/wiki/UTF-32
/wiki/Mac_OS_Romanian_encoding
/wiki/Code_page_1047
/wik

KeyboardInterrupt: 

## 2. Crawling an Entire Site

In order to avoid crawling the same page twice, it's extremely important that all internal links discovered are formatted consistently. Only links that are 'new' should be crawled.

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

pages = set()
def getLinks(pageUrl):
    global pages
    html = urlopen("http://en.wikipedia.org"+pageUrl)
    bsObj = BeautifulSoup(html)
    for link in bsObj.findAll("a", href=re.compile("^(/wiki/)")):
        if 'href' in link.attrs:
            if link.attrs['href'] not in pages:
                #We have encountered a new page
                newPage = link.attrs['href']
                print(newPage)
                pages.add(newPage)
                getLinks(newPage)

getLinks("")

## 3. Collecting Data Across an Entire Site

Let's look at how to build a scraper that collects the title, the first paragraph of content, and the link to edit the page (if available).

In [12]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

pages = set()
def getLinks(pageUrl):
    global pages
    html = urlopen("http://en.wikipedia.org"+pageUrl)
    bsObj = BeautifulSoup(html)
    try:
        print(bsObj.h1.get_text())
        print(bsObj.find(id="mw-content-text").findAll("p")[0])
        print(bsObj.find(id="ca-edit").find("span").find("a").attrs['href'])
    except AttributeError:
        print("This page is missing something! No worries though!")
    
    for link in bsObj.findAll("a", href=re.compile("^(/wiki/)")):
        if 'href' in link.attrs:
            if link.attrs['href'] not in pages:
                newPage = link.attrs['href']
                print("---------/n"+newPage)
                pages.add(newPage)
                getLinks(newPage)
getLinks("")
        

Main Page
<p>"<b><a href="/wiki/Home_(The_X-Files)" title="Home (The X-Files)">Home</a></b>", the second episode of the <a href="/wiki/The_X-Files_(season_4)" title="The X-Files (season 4)">fourth season</a> of the American <a href="/wiki/Science_fiction_on_television" title="Science fiction on television">science fiction</a> television series <i><a href="/wiki/The_X-Files" title="The X-Files">The X-Files</a></i>, originally aired on the <a href="/wiki/Fox_Broadcasting_Company" title="Fox Broadcasting Company">Fox</a> network on October<span class="nowrap"> </span>11, 1996. Directed by <a href="/wiki/Kim_Manners" title="Kim Manners">Kim Manners</a> and written by <a href="/wiki/Glen_Morgan" title="Glen Morgan">Glen Morgan</a> and <a href="/wiki/James_Wong_(filmmaker)" title="James Wong (filmmaker)">James Wong</a>, it is a "<a href="/wiki/Villain_of_the_week" title="Villain of the week">monster-of-the-week</a>" story, unconnected to the overarching <a href="/wiki/Mythology_of_The_X-File

KeyboardInterrupt: 

## 4. Crawling Across the Internet

A flexible set of Python functions that can be combined to perform a variety of different types of web scraping can be easily written in fewer than 50 lines of code.

In [18]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import datetime
import random

pages = set()
random.seed(datetime.datetime.now())

#Retrieves a list of all Internal links found on a page
def getInternalLinks(bsObj, includeUrl):
    internalLinks = []
    #Find all links that begin with a "/"
    for link in bsObj.findAll("a", href=re.compile("^(/|.*"+includeUrl+")")):
        if link.attrs['href'] is not None:
            if link.attrs['href'] not in internalLinks:
                internalLinks.append(link.attrs['href'])
    return internalLinks

def getExternalLinks(bsObj, excludeUrl):
    externalLinks = []
    for link in bsObj.findAll("a", href=re.compile("^(http|www)((?!"+excludeUrl+").)*$")):
        if link.attrs['href'] is not None:
            if link.attrs['href'] not in externalLinks:
                externalLinks.append(link.attrs['href'])
    return externalLinks

def splitAddress(address):
    addressParts = address.replace("http://","").split("/")
    return addressParts

def getRandomExternalLink(startingPage):
    html = urlopen(startingPage)
    bsObj = BeautifulSoup(html)
    externalLinks = getExternalLinks(bsObj, splitAddress(startingPage)[0])
    if len(externalLinks) == 0:
        internalLinks = getInternalLinks(startingPage)
        return getNextExternalLink(internalLinks[random.randint(0, len(internalLinks)-1)])
    else:
        return externalLinks[random.randint(0, len(externalLinks)-1)]

def followExternalOnly(startingSite):
    externalLink = getRandomExternalLink(startingSite)
    print("Random external link is: "+externalLink)
    followExternalOnly(externalLink)
    
followExternalOnly("http://oreilly.com")

Random external link is: https://twitter.com/oreillymedia
Random external link is: https://business.twitter.com/en/help/troubleshooting/how-twitter-ads-work.html?ref=web-twc-ao-gbl-adsinfo&utm_source=twc&utm_medium=web&utm_campaign=ao&utm_content=adsinfo
Random external link is: https://business.twitter.com/en/advertising.html
Random external link is: https://marketing.twitter.com/en/success-stories
Random external link is: https://data.twitter.com/

KeyboardInterrupt: 

Our goal now is to crawl an entire site for external links, and make a note of each one, we can modify the previously functions adding some factories like:

In [ ]:
allExtLinks = set()
allIntLinks = set()

def getAllExternalLinks(siteUrl):
    html = urlopen(siteUrl)
    bsObj = BeautifulSoup(html)
    internalLinks = getInternalLinks(bsObj,splitAddress(siteUrl)[0])
    externalLinks = getExternalLinks(bsObj,splitAddress(siteUrl)[0])
    for link in externalLinks:
        if link not in allExtLinks:
            allExtLinks.add(link)
    for link in internalLinks:
        if link not in allIntLinks:
            print("About to get link: "+link)
            allIntLinks.add(link)
            getAllExternalLinks(link)
            
getAllExternalLinks("http://oreilly.com")

## 5. Crawling with Scrapy

TBC..